<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab 3.02: Statistical Modeling and Model Validation

> Authors: Tim Book, Matt Brems

---

## Objective
The goal of this lab is to guide you through the modeling workflow. In this lesson, you will follow all best practices when slicing your data and validating your model. The goal of this lab is not necessarily to build the best model you can, but to build and evaluate a model, and interpret its results.

## Imports

In [2]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
# Muscle memory is important!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [5]:
# Read in the citibike data in the data folder in this repository.
citibike = pd.read_csv('./data/citibike_feb2014.csv')

## Explore the data
Use this space to familiarize yourself with the data.

Convince yourself there are no issues with the data. If you find any issues, clean them here.

In [7]:
citibike.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,21101,Subscriber,1991,1


In [15]:
# Birth year should be an integer
citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224736 entries, 0 to 224735
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             224736 non-null  int64  
 1   starttime                224736 non-null  object 
 2   stoptime                 224736 non-null  object 
 3   start station id         224736 non-null  int64  
 4   start station name       224736 non-null  object 
 5   start station latitude   224736 non-null  float64
 6   start station longitude  224736 non-null  float64
 7   end station id           224736 non-null  int64  
 8   end station name         224736 non-null  object 
 9   end station latitude     224736 non-null  float64
 10  end station longitude    224736 non-null  float64
 11  bikeid                   224736 non-null  int64  
 12  usertype                 224736 non-null  object 
 13  birth year               224736 non-null  object 
 14  gend

In [24]:
# How to change the 
citibike['birth year'].unique()

array(['1991', '1979', '1948', '1981', '1990', '1978', '1944', '1983',
       '1969', '1986', '1962', '1965', '1942', '1989', '1980', '1957',
       '1951', '1992', '1971', '1982', '1968', '1984', '\\N', '1956',
       '1987', '1985', '1996', '1975', '1988', '1974', '1972', '1959',
       '1973', '1977', '1976', '1953', '1993', '1970', '1963', '1967',
       '1966', '1960', '1961', '1994', '1958', '1955', '1946', '1964',
       '1900', '1995', '1954', '1952', '1949', '1947', '1941', '1938',
       '1950', '1945', '1997', '1934', '1940', '1939', '1936', '1943',
       '1935', '1937', '1922', '1932', '1907', '1926', '1899', '1901',
       '1917', '1910', '1933', '1921', '1927', '1913'], dtype=object)

In [26]:
citibike.shape

(224736, 15)

In [45]:
dog = citibike[citibike['birth year'] != '\\N']

In [55]:
dog.shape

(218019, 15)

In [54]:
count = 0
for n in dog['birth year']:
    count += int(n)
print(count/len(dog['birth year']))

1975.4975070980051


In [68]:
citibike['birth year'].replace('\\N', 1975)

0         1991
1         1979
2         1948
3         1981
4         1990
          ... 
224731    1976
224732    1985
224733    1968
224734    1982
224735    1960
Name: birth year, Length: 224736, dtype: object

In [ ]:
for n in citibike['birth year']:
    int(n)

In [67]:
citibike['birth year'] = citibike['birth year'].astype(float)

ValueError: could not convert string to float: '\\N'

In [65]:
citibike.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                  object
gender                       int64
dtype: object

In [32]:
citibike[citibike['birth year'] == '\\N']

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
31,664,2014-02-01 00:08:47,2014-02-01 00:19:51,237,E 11 St & 2 Ave,40.730473,-73.986724,349,Rivington St & Ridge St,40.718502,-73.983299,17540,Customer,\N,0
55,836,2014-02-01 00:16:10,2014-02-01 00:30:06,488,W 39 St & 9 Ave,40.756458,-73.993722,297,E 15 St & 3 Ave,40.734232,-73.986923,16731,Customer,\N,0
222,1277,2014-02-01 01:17:50,2014-02-01 01:39:07,469,Broadway & W 53 St,40.763441,-73.982681,336,Sullivan St & Washington Sq,40.730477,-73.999061,20728,Customer,\N,0
266,29906,2014-02-01 01:44:59,2014-02-01 10:03:25,294,Washington Square E,40.730494,-73.995721,368,Carmine St & 6 Ave,40.730386,-74.002150,18944,Customer,\N,0
293,2625,2014-02-01 01:56:32,2014-02-01 02:40:17,395,Bond St & Schermerhorn St,40.688070,-73.984106,395,Bond St & Schermerhorn St,40.688070,-73.984106,19782,Customer,\N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224385,280,2014-02-28 21:42:34,2014-02-28 21:47:14,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,382,University Pl & E 14 St,40.734927,-73.992005,15529,Customer,\N,0
224438,1106,2014-02-28 22:00:46,2014-02-28 22:19:12,423,W 54 St & 9 Ave,40.765849,-73.986905,385,E 55 St & 2 Ave,40.757973,-73.966033,18720,Customer,\N,0
224525,864,2014-02-28 22:29:52,2014-02-28 22:44:16,385,E 55 St & 2 Ave,40.757973,-73.966033,441,E 52 St & 2 Ave,40.756014,-73.967416,21399,Customer,\N,0
224536,683,2014-02-28 22:32:55,2014-02-28 22:44:18,385,E 55 St & 2 Ave,40.757973,-73.966033,441,E 52 St & 2 Ave,40.756014,-73.967416,17516,Customer,\N,0


In [12]:
# There are no null values
citibike.isna().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

### Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

### What numeric columns shouldn't be treated as numeric?

**Answer:**

### Dummify the `start station id` Variable

## Feature Engineering
Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected)
- Note: you will need to clean the data a bit.

## Split your data into train/test sets

Look at the size of your data. What is a good proportion for your split? **Justify your answer, considering the size of your data and the default split size in sklearn.**

Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `usertype`, `gender`, and the dummy variables you created from `start station id`. (Hint: You may find the Pandas `.drop()` method helpful here.) 

## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ to be the mean of our training `y` values.)

## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

## Evaluate your model
Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

## Citi Bike is attempting to market to people who they think will ride their bike for a long time. Based on your modeling, what types of individuals should Citi Bike market toward?